### AMIE rules before and after KG enrichment using TransE

In [1]:
def compare_rules_amie(rules_before, rules_after):
    with open(rules_before) as file:
        Rules_1 = file.read().strip().split('\n')
    with open(rules_after) as file:
        Rules_2 = file.read().strip().split('\n')
    rules_1 = list(filter(lambda x: x.startswith("?"), Rules_1))
    rules_2 = list(filter(lambda x: x.startswith("?"), Rules_2))
    Rules_1 = set(map(lambda x: x.split('\t')[0], rules_1))
    Rules_2 = set(map(lambda x: x.split('\t')[0], rules_2))
    
    new_rules = Rules_2-Rules_1
    
    dropped_rules = Rules_1-Rules_2
    
    same_rules = Rules_1.intersection(Rules_2)
    
    new_rules = sorted(filter(lambda x: x.split('\t')[0] in new_rules, rules_2),key=lambda t: float(t.split('\t')[3]), reverse=True)
    dropped_rules = sorted(filter(lambda x: x.split('\t')[0] in dropped_rules, rules_1),key=lambda t: float(t.split('\t')[3]), reverse=True)
    same_rules_1 = sorted(filter(lambda x: x.split('\t')[0] in same_rules, rules_1),key=lambda t: float(t.split('\t')[3]), reverse=True)
    same_rules_2 = sorted(filter(lambda x: x.split('\t')[0] in same_rules, rules_2),key=lambda t: float(t.split('\t')[3]), reverse=True)
    rules_1 = sorted(rules_1, key=lambda t: float(t.split('\t')[3]), reverse=True)
    rules_2 = sorted(rules_2, key=lambda t: float(t.split('\t')[3]), reverse=True)
    
    print(f'There were {len(Rules_1)} rules before, there are {len(Rules_2)} after enrichment')
    print()
    print(f'There are {len(same_rules)} same rules, {len(new_rules)} new rules, {len(dropped_rules)} dropped rules')
    
    #print('\nNew rules', new_rules)
    #print('\nDropped rules', dropped_rules)
    return same_rules_1, same_rules_2, rules_1, rules_2, new_rules, dropped_rules

## WN18RR

In [2]:
same_rules_amie_wn18rr_1, same_rules_amie_wn18rr_2, rules_amie_before_wn18rr, rules_amie_after_wn18rr, \
new_rules_amie_wn18rr, dropped_rules_amie_wn18rr = compare_rules_amie('AMIE/rules_wn18rr/rules.txt', 'TransAMIE/rules_wn18rr/rules.txt')

There were 34 rules before, there are 90 after enrichment

There are 24 same rules, 66 new rules, 10 dropped rules


In [3]:
same_rules_amie_wn18rr_1[:5]

['?b  _verb_group  ?a   => ?a  _verb_group  ?b\t0.931458699\t0.931458699\t0.980573543\t1060\t1138\t1081\t?b',
 '?b  _derivationally_related_form  ?a   => ?a  _derivationally_related_form  ?b\t0.932222783\t0.932222783\t0.95163008\t27701\t29715\t29109\t?b',
 '?b  _also_see  ?a   => ?a  _also_see  ?b\t0.637413395\t0.637413395\t0.883671291\t828\t1299\t937\t?b',
 '?a  _instance_hypernym  ?h  ?h  _synset_domain_topic_of  ?b   => ?a  _synset_domain_topic_of  ?b\t0.032092426\t0.395256917\t0.862068966\t100\t253\t116\t?a',
 '?a  _hypernym  ?h  ?h  _synset_domain_topic_of  ?b   => ?a  _synset_domain_topic_of  ?b\t0.178754814\t0.285056295\t0.809593023\t557\t1954\t688\t?a']

In [4]:
dropped_rules_amie_wn18rr[:5]

['?h  _synset_domain_topic_of  ?b  ?a  _verb_group  ?h   => ?a  _synset_domain_topic_of  ?b\t0.012516046\t0.375\t0.75\t39\t104\t52\t?a',
 '?g  _synset_domain_topic_of  ?b  ?g  _verb_group  ?a   => ?a  _synset_domain_topic_of  ?b\t0.012516046\t0.386138614\t0.75\t39\t101\t52\t?a',
 '?g  _hypernym  ?b  ?a  _member_of_domain_region  ?g   => ?a  _member_of_domain_region  ?b\t0.01191766\t0.021072797\t0.733333333\t11\t522\t15\t?b',
 '?g  _hypernym  ?b  ?a  _member_of_domain_usage  ?g   => ?a  _member_of_domain_usage  ?b\t0.028616852\t0.04400978\t0.72\t18\t409\t25\t?b',
 '?a  _has_part  ?h  ?b  _instance_hypernym  ?h   => ?a  _has_part  ?b\t0.029900332\t0.41025641\t0.52173913\t144\t351\t276\t?b']

In [5]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_wn18rr)))/len(dropped_rules_amie_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_wn18rr)))/len(dropped_rules_amie_wn18rr))

PCA: 0.4063505224
Std: 0.15840383379999998


In [6]:
new_rules_amie_wn18rr[-10:]

['?a  _member_meronym  ?b   => ?a  _similar_to  ?b\t0.460526316\t0.009140768\t0.707070707\t70\t7658\t99\t?b',
 '?a  _hypernym  ?b   => ?a  _similar_to  ?b\t0.414473684\t0.001971091\t0.692307692\t63\t31962\t91\t?b',
 '?a  _also_see  ?b   => ?a  _similar_to  ?b\t0.513157895\t0.06824147\t0.684210526\t78\t1143\t114\t?b',
 '?h  _hypernym  ?b  ?a  _member_of_domain_usage  ?h   => ?a  _member_of_domain_usage  ?b\t0.01610306\t0.02617801\t0.666666667\t10\t382\t15\t?b',
 '?a  _derivationally_related_form  ?b   => ?a  _similar_to  ?b\t0.513157895\t0.003055469\t0.527027027\t78\t25528\t148\t?b',
 '?a  _member_meronym  ?b   => ?a  _has_part  ?b\t0.015331462\t0.00927135\t0.360406091\t71\t7658\t197\t?b',
 '?a  _derivationally_related_form  ?b   => ?a  _member_meronym  ?b\t0.010446592\t0.003133814\t0.283687943\t80\t25528\t282\t?b',
 '?a  _hypernym  ?b   => ?a  _also_see  ?b\t0.058617673\t0.002096239\t0.201201201\t67\t31962\t333\t?b',
 '?a  _derivationally_related_form  ?b   => ?a  _has_part  ?b\t0.0166

In [7]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_wn18rr)))/len(new_rules_amie_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_wn18rr)))/len(new_rules_amie_wn18rr))

PCA: 0.8880675046818184
Std: 0.6548039246212123


In [8]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_wn18rr_1)))/len(same_rules_amie_wn18rr_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_wn18rr_1)))/len(same_rules_amie_wn18rr_1))

print()
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_wn18rr_2)))/len(same_rules_amie_wn18rr_2))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_wn18rr_2)))/len(same_rules_amie_wn18rr_2))

PCA: 0.5201082241249999
Std: 0.2835829905833334

PCA: 0.4736712131666667
Std: 0.226862331375


In [9]:
# Before
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_wn18rr)))/len(rules_amie_before_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_wn18rr)))/len(rules_amie_before_wn18rr))

PCA: 0.4866500765588235
Std: 0.24676559152941172


In [10]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_wn18rr)))/len(rules_amie_after_wn18rr))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_wn18rr)))/len(rules_amie_after_wn18rr))

PCA: 0.7775618269444449
Std: 0.5406861664222223


## Mutagenesis

In [11]:
same_rules_amie_mut_1, same_rules_amie_mut_2, rules_amie_before_mut, rules_amie_after_mut, \
new_rules_amie_mut, dropped_rules_amie_mut = compare_rules_amie('AMIE/rules_mutagenesis/rules.txt', 'TransAMIE/rules_mutagenesis/rules.txt')

There were 9 rules before, there are 17 after enrichment

There are 9 same rules, 8 new rules, 0 dropped rules


In [12]:
# Dropped rules, undefined
#sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_mut)))/len(dropped_rules_amie_mut)

In [13]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_mut)))/len(rules_amie_before_mut))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_mut)))/len(rules_amie_before_mut))

PCA: 0.5425176136666666
Std: 0.43713459811111105


In [14]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_mut)))/len(rules_amie_after_mut))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_mut)))/len(rules_amie_after_mut))

PCA: 0.5532036077058823
Std: 0.47268887935294107


In [15]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_mut_1)))/len(same_rules_amie_mut_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_mut_1)))/len(same_rules_amie_mut_1))
print()
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_mut_2)))/len(same_rules_amie_mut_2))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_mut_2)))/len(same_rules_amie_mut_2))

PCA: 0.5425176136666666
Std: 0.43713459811111105

PCA: 0.5449401478888889
Std: 0.47222185144444445


In [16]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_mut)))/len(new_rules_amie_mut))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_mut)))/len(new_rules_amie_mut))

PCA: 0.5625
Std: 0.47321428574999996


## Carcinogenesis

In [17]:
same_rules_amie_car_1, same_rules_amie_car_2, rules_amie_before_car, rules_amie_after_car, \
new_rules_amie_car, dropped_rules_amie_car = compare_rules_amie('AMIE/rules_carcinogenesis/rules.txt', 'TransAMIE/rules_carcinogenesis/rules.txt')

There were 185 rules before, there are 317 after enrichment

There are 147 same rules, 170 new rules, 38 dropped rules


In [18]:
# Before enrichment
print('PCA:', sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_car)))/len(rules_amie_before_car))
print('Std:', sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_car)))/len(rules_amie_before_car))

PCA: 0.7375154245621616
Std: 0.5162657981027027


In [19]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_car)))/len(rules_amie_after_car))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_car)))/len(rules_amie_after_car))

PCA: 0.695144739694006
Std: 0.4452791533312304


In [20]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_car)))/len(new_rules_amie_car))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_car)))/len(new_rules_amie_car))

PCA: 0.6813794434588235
Std: 0.4150670346941179


In [21]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_car)))/len(dropped_rules_amie_car))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_car)))/len(dropped_rules_amie_car))

PCA: 0.7535989299210527
Std: 0.5282689622105262


In [22]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_car_1)))/len(same_rules_amie_car_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_car_1)))/len(same_rules_amie_car_1))

PCA: 0.7333577837210885
Std: 0.5131629393537415


## FB15k

In [23]:
same_rules_amie_fb_1, same_rules_amie_fb_2, rules_amie_before_fb, rules_amie_after_fb, \
new_rules_amie_fb, dropped_rules_amie_fb = compare_rules_amie('AMIE/rules_fb15k/rules.txt', 'TransAMIE/rules_fb15k/rules.txt')

There were 7591 rules before, there are 10109 after enrichment

There are 7591 same rules, 4905 new rules, 0 dropped rules


In [24]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_fb)))/len(rules_amie_before_fb))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_fb)))/len(rules_amie_before_fb))

PCA: 0.5633267253446179
Std: 0.3071214942907393


In [25]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_fb)))/len(rules_amie_after_fb))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_fb)))/len(rules_amie_after_fb))

PCA: 0.5629404381041404
Std: 0.27093583009590555


In [26]:
# dropped rules, no dropped rules
#sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_fb)))/len(dropped_rules_amie_fb)

In [27]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_fb)))/len(new_rules_amie_fb))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_fb)))/len(new_rules_amie_fb))

PCA: 0.5460132989651448
Std: 0.1516606972941899


In [28]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_fb_1)))/len(same_rules_amie_fb_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_fb_1)))/len(same_rules_amie_fb_1))

PCA: 0.5633267253446179
Std: 0.3071214942907393


## YAGO3

In [29]:
same_rules_amie_yago_1, same_rules_amie_yago_2, rules_amie_before_yago, rules_amie_after_yago, \
new_rules_amie_yago, dropped_rules_amie_yago = compare_rules_amie('AMIE/rules_yago3/rules.txt', 'TransAMIE/rules_yago3/rules.txt')

There were 235 rules before, there are 310 after enrichment

There are 116 same rules, 194 new rules, 119 dropped rules


In [30]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_yago)))/len(rules_amie_before_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_yago)))/len(rules_amie_before_yago))

PCA: 0.3242538606127661
Std: 0.19154214194468083


In [31]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_yago)))/len(rules_amie_after_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_yago)))/len(rules_amie_after_yago))

PCA: 0.6707416839645169
Std: 0.6007938599645165


In [32]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_yago)))/len(new_rules_amie_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_yago)))/len(new_rules_amie_yago))

PCA: 0.8879264069587636
Std: 0.8583710295567017


In [33]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_yago)))/len(dropped_rules_amie_yago))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_yago)))/len(dropped_rules_amie_yago))

PCA: 0.2895052532268907
Std: 0.14846234643697478


In [34]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_yago_1)))/len(same_rules_amie_yago_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_yago_1)))/len(same_rules_amie_yago_1))

PCA: 0.35990113887931047
Std: 0.23573607009482755


## Open Bio Link

In [35]:
same_rules_amie_op_1, same_rules_amie_op_2, rules_amie_before_op, rules_amie_after_op, \
new_rules_amie_op, dropped_rules_amie_op = compare_rules_amie('AMIE/rules_openbiolink/rules.txt', 'TransAMIE/rules_openbiolink/rules.txt')

There were 713 rules before, there are 611 after enrichment

There are 510 same rules, 101 new rules, 203 dropped rules


In [36]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_op)))/len(rules_amie_before_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_op)))/len(rules_amie_before_op))

PCA: 0.31666675416129036
Std: 0.2804260248106593


In [37]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_op)))/len(rules_amie_after_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_op)))/len(rules_amie_after_op))

PCA: 0.3353037292291323
Std: 0.2905857795957448


In [38]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_op)))/len(dropped_rules_amie_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_op)))/len(dropped_rules_amie_op))

PCA: 0.24821895901477792
Std: 0.21569599121182248


In [39]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_op)))/len(new_rules_amie_op))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_op)))/len(new_rules_amie_op))

PCA: 0.6501759515148514
Std: 0.5803524768019802


In [40]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_op_1)))/len(same_rules_amie_op_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_op_1)))/len(same_rules_amie_op_1))

PCA: 0.3439116608568628
Std: 0.306191116615686


## DRKG

In [41]:
same_rules_amie_drkg_1, same_rules_amie_drkg_2, rules_amie_before_drkg, rules_amie_after_drkg, \
new_rules_amie_drkg, dropped_rules_amie_drkg = compare_rules_amie('AMIE/rules_drkg/rules.txt', 'TransAMIE/rules_drkg/rules.txt')

There were 2804 rules before, there are 2776 after enrichment

There are 1303 same rules, 1473 new rules, 1501 dropped rules


In [42]:
# Before enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_before_drkg)))/len(rules_amie_before_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_before_drkg)))/len(rules_amie_before_drkg))

PCA: 0.2894146629104849
Std: 0.16976895299358047


In [43]:
# After enrichment
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), rules_amie_after_drkg)))/len(rules_amie_after_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), rules_amie_after_drkg)))/len(rules_amie_after_drkg))

PCA: 0.28688603990669986
Std: 0.17173715055079278


In [44]:
# New rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), new_rules_amie_drkg)))/len(new_rules_amie_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), new_rules_amie_drkg)))/len(new_rules_amie_drkg))

PCA: 0.2713560277094369
Std: 0.1468777073951121


In [45]:
# Dropped rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), dropped_rules_amie_drkg)))/len(dropped_rules_amie_drkg))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), dropped_rules_amie_drkg)))/len(dropped_rules_amie_drkg))

PCA: 0.27349933227981377
Std: 0.14287335827581618


In [46]:
# Same rules
print('PCA:',sum(list(map(lambda x: float(x.split('\t')[3]), same_rules_amie_drkg_1)))/len(same_rules_amie_drkg_1))
print('Std:',sum(list(map(lambda x: float(x.split('\t')[2]), same_rules_amie_drkg_1)))/len(same_rules_amie_drkg_1))

PCA: 0.3077484397920187
Std: 0.20075152219646988
